# Notebook 02: Data Preprocessing

Systematic preprocessing pipeline to clean and prepare raw data for feature engineering.
Transform raw Ames Housing data into model-ready format with proper feature types and zero missing values.

## 1. Data Loading and Initial Processing

### 1.1 Dataset Import

Import training and test datasets, create combined dataset for consistent preprocessing across both sets.

In [1]:
# Load required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Load the datasets
df_train = pd.read_csv('../data/raw/train.csv')
df_test = pd.read_csv('../data/raw/test.csv')

print("Dataset Import Validation:")
print(f"Training data: {df_train.shape}")
print(f"Test data: {df_test.shape}")

# Create combined dataset for consistent feature processing
df_combined = pd.concat([
    df_train.drop('SalePrice', axis=1),
    df_test
], ignore_index=True)
df_combined['dataset_source'] = ['train']*len(df_train) + ['test']*len(df_test)

print(f"Combined dataset: {df_combined.shape}")

print("\nData Types in Combined Dataset:")
print(df_combined.drop('dataset_source', axis=1).dtypes.value_counts())

Dataset Import Validation:
Training data: (1460, 81)
Test data: (1459, 80)
Combined dataset: (2919, 81)

Data Types in Combined Dataset:
object     43
int64      26
float64    11
Name: count, dtype: int64


### 1.2 Parser Integration Setup

In [2]:
# Setup data description parser for domain knowledge
from data_description_parser import (
    load_feature_descriptions,
    quick_feature_lookup,
    get_categorical_features,
    get_numerical_features,
)

# Load official documentation
feature_descriptions = load_feature_descriptions()
print("Parser Integration Setup:")

# Get feature classifications for preprocessing
categorical_features = get_categorical_features(feature_descriptions)
numerical_features = get_numerical_features(feature_descriptions)

print(f"Categorical features identified: {len(categorical_features)}")
print(f"Numerical features identified: {len(numerical_features)}")

Parser Integration Setup:
Categorical features identified: 46
Numerical features identified: 33


Parser confirms 46 categorical and 33 numerical features with 3 critical misclassifications requiring correction.
Official documentation provides guidance for every preprocessing decision.

## 2. Feature Classification Correction

Correct feature type misclassifications before proceeding with missing data treatment or encoding.
Ordinal features require different treatment than continuous numerical variables.

### 2.1 Ordinal Feature Correction

Convert three features (OverallQual, OverallCond, MSSubClass) from integers to proper categorical types.
This correction ensures ordinal relationships are preserved during encoding.

In [3]:
# Correct misclassified ordinal features identified in Notebook 01
ordinal_features = ['OverallQual', 'OverallCond', 'MSSubClass']

print("Ordinal Feature Correction:")

# Show current state before correction
print(f"\nBefore correction:")
for feature in ordinal_features:
    dtype = df_combined[feature].dtype
    unique_vals = sorted(df_combined[feature].unique())
    print(f"  {feature}: {dtype} with {len(unique_vals)} unique values: {unique_vals}")

# Convert to ordered categorical for combined dataset
print(f"\nApplying corrections:")
for feature in ordinal_features:
    if feature == 'MSSubClass':
        # MSSubClass: dwelling type categories
        print(f"  {feature}: Converting to unordered categorical (dwelling types)")
        df_combined[feature] = df_combined[feature].astype('category')
    else:
        # OverallQual and OverallCond: 1-10 quality scales
        print(f"  {feature}: Converting to ordered categorical (quality scale)")
        df_combined[feature] = pd.Categorical(df_combined[feature],
                                            categories=sorted(df_combined[feature].unique()),
                                            ordered=True)

print(f"\nAfter correction:")
for feature in ordinal_features:
    dtype = df_combined[feature].dtype
    is_ordered = hasattr(df_combined[feature], 'cat') and df_combined[feature].cat.ordered
    print(f"  {feature}: {dtype} (ordered: {is_ordered})")
    
    # Show categories for verification
    if hasattr(df_combined[feature], 'cat'):
        categories = list(df_combined[feature].cat.categories)
        print(f"    Categories: {categories}")

Ordinal Feature Correction:

Before correction:
  OverallQual: int64 with 10 unique values: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]
  OverallCond: int64 with 9 unique values: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)]
  MSSubClass: int64 with 16 unique values: [np.int64(20), np.int64(30), np.int64(40), np.int64(45), np.int64(50), np.int64(60), np.int64(70), np.int64(75), np.int64(80), np.int64(85), np.int64(90), np.int64(120), np.int64(150), np.int64(160), np.int64(180), np.int64(190)]

Applying corrections:
  OverallQual: Converting to ordered categorical (quality scale)
  OverallCond: Converting to ordered categorical (quality scale)
  MSSubClass: Converting to unordered categorical (dwelling types)

After correction:
  OverallQual: category (ordered: True)
    Categories: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  OverallCond: 

Converted 3 misclassified features to proper categorical types. OverallQual and OverallCond now preserve ordinal relationships (1-10 scales), while MSSubClass represents dwelling type categories.

## 3. Outlier Treatment

### 3.1 Outlier Removal



In [4]:

# Remove data quality outliers identified in Notebook 01
outlier_ids = [524, 1299]  # Partial sales of incomplete luxury properties

print(f"Outlier IDs to remove: {outlier_ids}")

# Show outlier characteristics before removal
outlier_analysis = df_train[df_combined['Id'].isin(outlier_ids)]
print(f"\nOutlier characteristics:")
for idx, row in outlier_analysis.iterrows():
    print(f"  ID {row['Id']}: {row['GrLivArea']:.0f} sqft, ${row['SalePrice']:,}, OverallQual={row['OverallQual']}")

# Remove outliers from both datasets
before_train_count = len(df_train)
before_combined_count = len(df_combined)

df_train = df_train[~df_train['Id'].isin(outlier_ids)].reset_index(drop=True)
df_combined = df_combined[~df_combined['Id'].isin(outlier_ids)].reset_index(drop=True)

after_train_count = len(df_train)
after_combined_count = len(df_combined)

print(f"\nOutlier removal impact:")
print(f"  Training data: {before_train_count} → {after_train_count} samples")
print(f"  Combined data: {before_combined_count} → {after_combined_count} samples")
print(f"  Removed: {before_train_count - after_train_count} samples ({((before_train_count - after_train_count) / before_train_count) * 100:.2f}%)")



Outlier IDs to remove: [524, 1299]

Outlier characteristics:
  ID 524: 4676 sqft, $184,750, OverallQual=10
  ID 1299: 5642 sqft, $160,000, OverallQual=10

Outlier removal impact:
  Training data: 1460 → 1458 samples
  Combined data: 2919 → 2917 samples
  Removed: 2 samples (0.14%)


Targeted outlier removal eliminates two data quality issues while preserving 99.86% of training data.

### 3.2 Impact Assessment

Assess impact of outlier removal on target variable

In [5]:
# Calculate statistics before outlier removal for comparison
df_train_original = pd.read_csv('../data/raw/train.csv')  # Reload original for comparison
df_train_original['SalePrice_log'] = np.log1p(df_train_original['SalePrice'])

print("Outlier Removal Impact Assessment:")

# Before/after comparison for SalePrice statistics
print(f"SalePrice statistics comparison:")
print(f"  Mean: ${df_train_original['SalePrice'].mean():,.0f} → ${df_train['SalePrice'].mean():,.0f}")
print(f"  Median: ${df_train_original['SalePrice'].median():,.0f} → ${df_train['SalePrice'].median():,.0f}")
print(f"  Std: ${df_train_original['SalePrice'].std():,.0f} → ${df_train['SalePrice'].std():,.0f}")
print(f"  Skewness: {df_train_original['SalePrice'].skew():.4f} → {df_train['SalePrice'].skew():.4f}")


Outlier Removal Impact Assessment:
SalePrice statistics comparison:
  Mean: $180,921 → $180,933
  Median: $163,000 → $163,000
  Std: $79,443 → $79,495
  Skewness: 1.8829 → 1.8813


Outlier removal had minimal impact on price distribution with mean/median stable, std unchanged (~$79k), and skewness slightly improved (1.8829→1.8813)

## 4. Missing Data Treatment

Address 34 features with missing values using a systematic three-tier approach.
Use parser guidance to distinguish between architectural absence and genuine missing data.

### 4.1 Missing Data Analysis

Analyze missing data patterns to organize features for systematic treatment.
This analysis guides our treatment strategy by categorizing different types of missingness.

In [6]:
# Get all features with missing data from combined dataset
missing_data = df_combined.drop('dataset_source', axis=1).isnull().sum()
missing_features = missing_data[missing_data > 0].sort_values(ascending=False)

print("Missing Data Overview:")
print(f"Total features with missing data: {len(missing_features)}")
print(f"Total missing values: {missing_features.sum()}")

print(f"\nAll missing features:")
for feature, count in missing_features.items():
    pct = (count / len(df_combined)) * 100
    print(f"  {feature}: {count} ({pct:.1f}%)")

Missing Data Overview:
Total features with missing data: 34
Total missing values: 15700

All missing features:
  PoolQC: 2908 (99.7%)
  MiscFeature: 2812 (96.4%)
  Alley: 2719 (93.2%)
  Fence: 2346 (80.4%)
  MasVnrType: 1766 (60.5%)
  FireplaceQu: 1420 (48.7%)
  LotFrontage: 486 (16.7%)
  GarageFinish: 159 (5.5%)
  GarageQual: 159 (5.5%)
  GarageCond: 159 (5.5%)
  GarageYrBlt: 159 (5.5%)
  GarageType: 157 (5.4%)
  BsmtExposure: 82 (2.8%)
  BsmtCond: 82 (2.8%)
  BsmtQual: 81 (2.8%)
  BsmtFinType2: 80 (2.7%)
  BsmtFinType1: 79 (2.7%)
  MasVnrArea: 23 (0.8%)
  MSZoning: 4 (0.1%)
  BsmtFullBath: 2 (0.1%)
  BsmtHalfBath: 2 (0.1%)
  Functional: 2 (0.1%)
  Utilities: 2 (0.1%)
  GarageArea: 1 (0.0%)
  GarageCars: 1 (0.0%)
  Electrical: 1 (0.0%)
  KitchenQual: 1 (0.0%)
  TotalBsmtSF: 1 (0.0%)
  BsmtUnfSF: 1 (0.0%)
  BsmtFinSF2: 1 (0.0%)
  BsmtFinSF1: 1 (0.0%)
  Exterior2nd: 1 (0.0%)
  Exterior1st: 1 (0.0%)
  SaleType: 1 (0.0%)


### 4.2 Systematic Feature Treatment

Analyze each feature to determine if missing data represents architectural absence or measurement gaps, then apply appropriate treatment strategies. If missing data represents architectural absence, fill with 'None' instead of 'NA'.


#### Function to Analyze Feature Missing Data

```python

In [7]:
def analyze_feature_missing_data(feature):
    """
    Analyze a single feature's missing data patterns and distribution.
    Displays missing value counts per dataset and distribution stats.

    Parameters:
    feature: Feature name to analyze
    """
    print(f"\n{feature}")
    print("-" * 60)

    # Total missing in combined
    total_missing = df_combined[feature].isnull().sum()
    total_pct = (total_missing / len(df_combined)) * 100
    print(f"Total Missing: {total_missing} values ({total_pct:.1f}%)")

    # Missing values by dataset
    for name, df in zip(['Train', 'Test', 'Combined'], [df_train, df_test, df_combined]):
        missing = df[feature].isnull().sum()
        pct = (missing / len(df)) * 100
        print(f"  {name}: {missing} missing ({pct:.1f}%)")

    # Feature description if available
    if 'feature_descriptions' in globals():
        quick_feature_lookup(feature, feature_descriptions)

    dtype = df_combined[feature].dtype

    # Categorical
    if dtype == 'object' or pd.api.types.is_categorical_dtype(dtype):
        print("\nData type: Categorical")

        for name, df in zip(['Train', 'Test', 'Combined'], [df_train, df_test, df_combined]):
            print(f"\n{name} Value Distribution:")
            counts = df[feature].value_counts()
            for val, cnt in counts.items():
                pct = cnt / counts.sum() * 100
                print(f"  {val}: {cnt} ({pct:.1f}%)")

    else:
        print("\nData type: Numerical")
        for name, df in zip(['Train', 'Test', 'Combined'], [df_train, df_test, df_combined]):
            values = df[feature].dropna()
            if len(values) == 0:
                continue
            print(f"\n{name} Stats:")
            print(f"  Range: {values.min():.1f} – {values.max():.1f}")
            print(f"  Mean: {values.mean():.1f}, Median: {values.median():.1f}")
            zeros = (values == 0).sum()
            if zeros > 0:
                zero_pct = zeros / len(values) * 100
                print(f"  Zero values: {zeros} ({zero_pct:.1f}%)")



#### PoolQC

In [8]:
analyze_feature_missing_data('PoolQC')


PoolQC
------------------------------------------------------------
Total Missing: 2908 values (99.7%)
  Train: 1452 missing (99.6%)
  Test: 1456 missing (99.8%)
  Combined: 2908 missing (99.7%)
Feature: PoolQC
Description: Pool quality
Type: Categorical

Categories:
  Ex: Excellent
  Gd: Good
  TA: Average/Typical
  Fa: Fair
  NA: No Pool
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Ex: 2 (33.3%)
  Fa: 2 (33.3%)
  Gd: 2 (33.3%)

Test Value Distribution:
  Ex: 2 (66.7%)
  Gd: 1 (33.3%)

Combined Value Distribution:
  Ex: 4 (44.4%)
  Gd: 3 (33.3%)
  Fa: 2 (22.2%)


In [9]:
# Filling missing PoolQC with None
df_combined['PoolQC'].fillna('None', inplace=True)
print('Missing values in PoolQC after filling with None:', df_combined['PoolQC'].isnull().sum())

Missing values in PoolQC after filling with None: 0


#### MiscFeature

In [10]:
analyze_feature_missing_data('MiscFeature')


MiscFeature
------------------------------------------------------------
Total Missing: 2812 values (96.4%)
  Train: 1404 missing (96.3%)
  Test: 1408 missing (96.5%)
  Combined: 2812 missing (96.4%)
Feature: MiscFeature
Description: Miscellaneous feature not covered in other categories
Type: Categorical

Categories:
  Elev: Elevator
  Gar2: 2nd Garage (if not described in garage section)
  Othr: Other
  Shed: Shed (over 100 SF)
  TenC: Tennis Court
  NA: None
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Shed: 49 (90.7%)
  Gar2: 2 (3.7%)
  Othr: 2 (3.7%)
  TenC: 1 (1.9%)

Test Value Distribution:
  Shed: 46 (90.2%)
  Gar2: 3 (5.9%)
  Othr: 2 (3.9%)

Combined Value Distribution:
  Shed: 95 (90.5%)
  Gar2: 5 (4.8%)
  Othr: 4 (3.8%)
  TenC: 1 (1.0%)


In [11]:
# Filling missing MiscFeature with None
df_combined['MiscFeature'].fillna('None', inplace=True)
print('Missing values in MiscFeature after filling with None:', df_combined['MiscFeature'].isnull().sum())

Missing values in MiscFeature after filling with None: 0


#### Alley

In [12]:
analyze_feature_missing_data('Alley')


Alley
------------------------------------------------------------
Total Missing: 2719 values (93.2%)
  Train: 1367 missing (93.8%)
  Test: 1352 missing (92.7%)
  Combined: 2719 missing (93.2%)
Feature: Alley
Description: Type of alley access to property
Type: Categorical

Categories:
  Grvl: Gravel
  Pave: Paved
  NA: No alley access
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Grvl: 50 (54.9%)
  Pave: 41 (45.1%)

Test Value Distribution:
  Grvl: 70 (65.4%)
  Pave: 37 (34.6%)

Combined Value Distribution:
  Grvl: 120 (60.6%)
  Pave: 78 (39.4%)


In [13]:
# Filling missing Alley with None
df_combined['Alley'].fillna('None', inplace=True)
print('Missing values in Alley after filling with None:', df_combined['Alley'].isnull().sum())

Missing values in Alley after filling with None: 0


#### Fence

In [14]:
analyze_feature_missing_data('Fence')


Fence
------------------------------------------------------------
Total Missing: 2346 values (80.4%)
  Train: 1177 missing (80.7%)
  Test: 1169 missing (80.1%)
  Combined: 2346 missing (80.4%)
Feature: Fence
Description: Fence quality
Type: Categorical

Categories:
  GdPrv: Good Privacy
  MnPrv: Minimum Privacy
  GdWo: Good Wood
  MnWw: Minimum Wood/Wire
  NA: No Fence
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  MnPrv: 157 (55.9%)
  GdPrv: 59 (21.0%)
  GdWo: 54 (19.2%)
  MnWw: 11 (3.9%)

Test Value Distribution:
  MnPrv: 172 (59.3%)
  GdPrv: 59 (20.3%)
  GdWo: 58 (20.0%)
  MnWw: 1 (0.3%)

Combined Value Distribution:
  MnPrv: 329 (57.6%)
  GdPrv: 118 (20.7%)
  GdWo: 112 (19.6%)
  MnWw: 12 (2.1%)


In [15]:
# Filling missing Fence with None
df_combined['Fence'].fillna('None', inplace=True)
print('Missing values in Fence after filling with None:', df_combined['Fence'].isnull().sum())

Missing values in Fence after filling with None: 0


#### MasVnrType

In [16]:
analyze_feature_missing_data('MasVnrType')


MasVnrType
------------------------------------------------------------
Total Missing: 1766 values (60.5%)
  Train: 872 missing (59.8%)
  Test: 894 missing (61.3%)
  Combined: 1766 missing (60.5%)
Feature: MasVnrType
Description: Masonry veneer type
Type: Categorical

Categories:
  BrkCmn: Brick Common
  BrkFace: Brick Face
  CBlock: Cinder Block
  None: None
  Stone: Stone
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  BrkFace: 445 (75.9%)
  Stone: 126 (21.5%)
  BrkCmn: 15 (2.6%)

Test Value Distribution:
  BrkFace: 434 (76.8%)
  Stone: 121 (21.4%)
  BrkCmn: 10 (1.8%)

Combined Value Distribution:
  BrkFace: 879 (76.4%)
  Stone: 247 (21.5%)
  BrkCmn: 25 (2.2%)


In [17]:
# Filling missing MasVnrType with None
df_combined['MasVnrType'].fillna('None', inplace=True)
print('Missing values in MasVnrType after filling with None:', df_combined['MasVnrType'].isnull().sum())

Missing values in MasVnrType after filling with None: 0


#### FireplaceQu

In [18]:
analyze_feature_missing_data('FireplaceQu')


FireplaceQu
------------------------------------------------------------
Total Missing: 1420 values (48.7%)
  Train: 690 missing (47.3%)
  Test: 730 missing (50.0%)
  Combined: 1420 missing (48.7%)
Feature: FireplaceQu
Description: Fireplace quality
Type: Categorical

Categories:
  Ex: Excellent - Exceptional Masonry Fireplace
  Gd: Good - Masonry Fireplace in main level
  TA: Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
  Fa: Fair - Prefabricated Fireplace in basement
  Po: Poor - Ben Franklin Stove
  NA: No Fireplace
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Gd: 378 (49.2%)
  TA: 313 (40.8%)
  Fa: 33 (4.3%)
  Ex: 24 (3.1%)
  Po: 20 (2.6%)

Test Value Distribution:
  Gd: 364 (49.9%)
  TA: 279 (38.3%)
  Fa: 41 (5.6%)
  Po: 26 (3.6%)
  Ex: 19 (2.6%)

Combined Value Distribution:
  Gd: 742 (49.6%)
  TA: 592 (39.5%)
  Fa: 74 (4.9%)
  Po: 46 (3.1%)
  Ex: 43 (2.9%)


In [19]:
# Filling missing FireplaceQu with None
df_combined['FireplaceQu'].fillna('None', inplace=True)
print('Missing values in FireplaceQu after filling with None:', df_combined['FireplaceQu'].isnull().sum())

Missing values in FireplaceQu after filling with None: 0


#### LotFrontage

In [20]:
analyze_feature_missing_data('LotFrontage')



LotFrontage
------------------------------------------------------------
Total Missing: 486 values (16.7%)
  Train: 259 missing (17.8%)
  Test: 227 missing (15.6%)
  Combined: 486 missing (16.7%)
Feature: LotFrontage
Description: Linear feet of street connected to property
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 21.0 – 313.0
  Mean: 69.8, Median: 69.0

Test Stats:
  Range: 21.0 – 200.0
  Mean: 68.6, Median: 67.0

Combined Stats:
  Range: 21.0 – 313.0
  Mean: 69.2, Median: 68.0


In [21]:
# Fill missing LotFrontage using neighborhood medians from training data only
# Fallback: global median from training data (no leakage)

# Compute neighborhood-level medians from training data
train_medians = df_train.groupby('Neighborhood')['LotFrontage'].median()

# Compute global median from training data
global_median = df_train['LotFrontage'].median()

# Fill missing values in combined dataset
df_combined['LotFrontage'] = df_combined.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(train_medians.get(x.name, global_median))
)

missing_count = df_combined['LotFrontage'].isnull().sum()
print(f"Missing values in LotFrontage after filling with training medians: {missing_count}")



Missing values in LotFrontage after filling with training medians: 0


### GarageFinish

In [22]:
analyze_feature_missing_data('GarageFinish')


GarageFinish
------------------------------------------------------------
Total Missing: 159 values (5.5%)
  Train: 81 missing (5.6%)
  Test: 78 missing (5.3%)
  Combined: 159 missing (5.5%)
Feature: GarageFinish
Description: Interior finish of the garage
Type: Categorical

Categories:
  Fin: Finished
  RFn: Rough Finished
  Unf: Unfinished
  NA: No Garage
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Unf: 605 (43.9%)
  RFn: 422 (30.6%)
  Fin: 350 (25.4%)

Test Value Distribution:
  Unf: 625 (45.3%)
  RFn: 389 (28.2%)
  Fin: 367 (26.6%)

Combined Value Distribution:
  Unf: 1230 (44.6%)
  RFn: 811 (29.4%)
  Fin: 717 (26.0%)


In [23]:
# Fill missing GarageFinish with None
df_combined['GarageFinish'].fillna('None', inplace=True)
print('Missing values in GarageFinish after filling with None:', df_combined['GarageFinish'].isnull().sum())

Missing values in GarageFinish after filling with None: 0


#### GarageQual

In [24]:
analyze_feature_missing_data('GarageQual')


GarageQual
------------------------------------------------------------
Total Missing: 159 values (5.5%)
  Train: 81 missing (5.6%)
  Test: 78 missing (5.3%)
  Combined: 159 missing (5.5%)
Feature: GarageQual
Description: Garage quality
Type: Categorical

Categories:
  Ex: Excellent
  Gd: Good
  TA: Typical/Average
  Fa: Fair
  Po: Poor
  NA: No Garage
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  TA: 1309 (95.1%)
  Fa: 48 (3.5%)
  Gd: 14 (1.0%)
  Ex: 3 (0.2%)
  Po: 3 (0.2%)

Test Value Distribution:
  TA: 1293 (93.6%)
  Fa: 76 (5.5%)
  Gd: 10 (0.7%)
  Po: 2 (0.1%)

Combined Value Distribution:
  TA: 2602 (94.3%)
  Fa: 124 (4.5%)
  Gd: 24 (0.9%)
  Po: 5 (0.2%)
  Ex: 3 (0.1%)


In [25]:
# Fill missing GarageQual with None
df_combined['GarageQual'].fillna('None', inplace=True)
print('Missing values in GarageQual after filling with None:', df_combined['GarageQual'].isnull().sum())

Missing values in GarageQual after filling with None: 0


#### GarageCond

In [26]:
analyze_feature_missing_data('GarageCond')


GarageCond
------------------------------------------------------------
Total Missing: 159 values (5.5%)
  Train: 81 missing (5.6%)
  Test: 78 missing (5.3%)
  Combined: 159 missing (5.5%)
Feature: GarageCond
Description: Garage condition
Type: Categorical

Categories:
  Ex: Excellent
  Gd: Good
  TA: Typical/Average
  Fa: Fair
  Po: Poor
  NA: No Garage
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  TA: 1324 (96.2%)
  Fa: 35 (2.5%)
  Gd: 9 (0.7%)
  Po: 7 (0.5%)
  Ex: 2 (0.1%)

Test Value Distribution:
  TA: 1328 (96.2%)
  Fa: 39 (2.8%)
  Po: 7 (0.5%)
  Gd: 6 (0.4%)
  Ex: 1 (0.1%)

Combined Value Distribution:
  TA: 2652 (96.2%)
  Fa: 74 (2.7%)
  Gd: 15 (0.5%)
  Po: 14 (0.5%)
  Ex: 3 (0.1%)


In [27]:
# Fill missing GarageCond with None
df_combined['GarageCond'].fillna('None', inplace=True)
print('Missing values in GarageCond after filling with None:', df_combined['GarageCond'].isnull().sum())

Missing values in GarageCond after filling with None: 0


#### GarageType

In [28]:
analyze_feature_missing_data('GarageType')


GarageType
------------------------------------------------------------
Total Missing: 157 values (5.4%)
  Train: 81 missing (5.6%)
  Test: 76 missing (5.2%)
  Combined: 157 missing (5.4%)
Feature: GarageType
Description: Garage location
Type: Categorical

Categories:
  2Types: More than one type of garage
  Attchd: Attached to home
  Basment: Basement Garage
  BuiltIn: Built-In (Garage part of house - typically has room above garage)
  CarPort: Car Port
  Detchd: Detached from home
  NA: No Garage
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Attchd: 869 (63.1%)
  Detchd: 387 (28.1%)
  BuiltIn: 87 (6.3%)
  Basment: 19 (1.4%)
  CarPort: 9 (0.7%)
  2Types: 6 (0.4%)

Test Value Distribution:
  Attchd: 853 (61.7%)
  Detchd: 392 (28.3%)
  BuiltIn: 98 (7.1%)
  Basment: 17 (1.2%)
  2Types: 17 (1.2%)
  CarPort: 6 (0.4%)

Combined Value Distribution:
  Attchd: 1722 (62.4%)
  Detchd: 779 (28.2%)
  BuiltIn: 185 (6.7%)
  Basment

In [29]:
# Fill missing GarageType with None
df_combined['GarageType'].fillna('None', inplace=True)
print('Missing values in GarageType after filling with None:', df_combined['GarageType'].isnull().sum())

Missing values in GarageType after filling with None: 0


#### GarageArea

In [30]:
analyze_feature_missing_data('GarageArea')


GarageArea
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: GarageArea
Description: Size of garage in square feet
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 0.0 – 1390.0
  Mean: 472.1, Median: 479.5
  Zero values: 81 (5.6%)

Test Stats:
  Range: 0.0 – 1488.0
  Mean: 472.8, Median: 480.0
  Zero values: 76 (5.2%)

Combined Stats:
  Range: 0.0 – 1488.0
  Mean: 472.4, Median: 480.0
  Zero values: 157 (5.4%)


In [31]:
# Fill missing GarageArea with 0
df_combined['GarageArea'].fillna(0, inplace=True)
print('Missing values in GarageArea after filling with 0:', df_combined['GarageArea'].isnull().sum())

Missing values in GarageArea after filling with 0: 0


#### GarageYrBlt

In [32]:
analyze_feature_missing_data('GarageYrBlt')


GarageYrBlt
------------------------------------------------------------
Total Missing: 159 values (5.5%)
  Train: 81 missing (5.6%)
  Test: 78 missing (5.3%)
  Combined: 159 missing (5.5%)
Feature: GarageYrBlt
Description: Year garage was built
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 1900.0 – 2010.0
  Mean: 1978.5, Median: 1980.0

Test Stats:
  Range: 1895.0 – 2207.0
  Mean: 1977.7, Median: 1979.0

Combined Stats:
  Range: 1895.0 – 2207.0
  Mean: 1978.1, Median: 1979.0


In [33]:
# Change 2207 to 2007
mask = df_combined['GarageYrBlt'] == 2207

print("Before correction:")
display(
    df_combined.loc[mask, ['Id', 'YearBuilt', 'GarageYrBlt']]
    .style.hide(axis='index')
)

ids_to_fix = df_combined.loc[mask, 'Id']

df_combined.loc[mask, 'GarageYrBlt'] = 2007

print("After correction:")
display(
    df_combined.loc[df_combined['Id'].isin(ids_to_fix), ['Id', 'YearBuilt', 'GarageYrBlt']]
    .style.hide(axis='index')
)



Before correction:


Id,YearBuilt,GarageYrBlt
2593,2006,2207.000000


After correction:


Id,YearBuilt,GarageYrBlt
2593,2006,2007.000000


In [34]:
# Find garages with other features, but no GarageYrBlt
has_garage_missing_year = (df_combined['GarageType'] != 'None') & df_combined['GarageYrBlt'].isnull()

garage_columns = [
    'Id', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'GarageCars', 'GarageArea', 'GarageYrBlt',
    'YearBuilt', 'YearRemodAdd'
]

display(
    df_combined.loc[has_garage_missing_year, garage_columns]
        .style.hide(axis='index')
)


Id,GarageType,GarageFinish,GarageQual,GarageCond,GarageCars,GarageArea,GarageYrBlt,YearBuilt,YearRemodAdd
2127,Detchd,None,None,None,1.000000,360.000000,nan,1910,1983
2577,Detchd,None,None,None,nan,0.000000,nan,1923,1999


In [35]:
# Fix inconsistent garage data for Id 2127: garage exists (has area) but some details are missing
df_combined.loc[df_combined['Id'] == 2127, 'GarageYrBlt'] = df_combined.loc[df_combined['Id'] == 2127, 'YearBuilt']  # Set GarageYrBlt to same year as house (1910)
df_combined.loc[df_combined['Id'] == 2127, 'GarageFinish'] = 'Unf'  # Set GarageFinish to 'Unf' (unfinished)
df_combined.loc[df_combined['Id'] == 2127, 'GarageQual'] = 'TA'     # Set GarageQual to 'TA' (typical/average quality)
df_combined.loc[df_combined['Id'] == 2127, 'GarageCond'] = 'TA'     # Set GarageCond to 'TA' (typical/average condition)

In [36]:
# ID 2577: No garage 
df_combined.loc[df_combined['Id'] == 2577, 'GarageType'] = 'None'            
df_combined.loc[df_combined['Id'] == 2577, 'GarageCars'] = 0               
df_combined.loc[df_combined['Id'] == 2577, 'GarageYrBlt'] = 0              


In [37]:
# Display garages after fixing
display(
    df_combined.loc[df_combined['Id'].isin([2127, 2577]), garage_columns]
        .style.hide(axis='index')
)

Id,GarageType,GarageFinish,GarageQual,GarageCond,GarageCars,GarageArea,GarageYrBlt,YearBuilt,YearRemodAdd
2127,Detchd,Unf,TA,TA,1.000000,360.000000,1910.000000,1910,1983
2577,None,None,None,None,0.000000,0.000000,0.000000,1923,1999


In [38]:
# Fill missing GarageYrBlt with 0 for the rest
df_combined['GarageYrBlt'].fillna(0, inplace=True)
print('Missing values in GarageYrBlt after filling with 0:', df_combined['GarageYrBlt'].isnull().sum())

Missing values in GarageYrBlt after filling with 0: 0


#### BsmtExposure

In [39]:
analyze_feature_missing_data('BsmtExposure')


BsmtExposure
------------------------------------------------------------
Total Missing: 82 values (2.8%)
  Train: 38 missing (2.6%)
  Test: 44 missing (3.0%)
  Combined: 82 missing (2.8%)
Feature: BsmtExposure
Description: Refers to walkout or garden level walls
Type: Categorical

Categories:
  Gd: Good Exposure
  Av: Average Exposure (split levels or foyers typically score average or above)
  Mn: Mimimum Exposure
  No: No Exposure
  NA: No Basement
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  No: 953 (67.1%)
  Av: 221 (15.6%)
  Gd: 132 (9.3%)
  Mn: 114 (8.0%)

Test Value Distribution:
  No: 951 (67.2%)
  Av: 197 (13.9%)
  Gd: 142 (10.0%)
  Mn: 125 (8.8%)

Combined Value Distribution:
  No: 1904 (67.2%)
  Av: 418 (14.7%)
  Gd: 274 (9.7%)
  Mn: 239 (8.4%)


In [40]:
# Fill missing BsmtExposure with None
df_combined['BsmtExposure'].fillna('None', inplace=True)
print('Missing values in BsmtExposure after filling with None:', df_combined['BsmtExposure'].isnull().sum())

Missing values in BsmtExposure after filling with None: 0


#### BsmtCond

In [41]:
analyze_feature_missing_data('BsmtCond')


BsmtCond
------------------------------------------------------------
Total Missing: 82 values (2.8%)
  Train: 37 missing (2.5%)
  Test: 45 missing (3.1%)
  Combined: 82 missing (2.8%)
Feature: BsmtCond
Description: Evaluates the general condition of the basement
Type: Categorical

Categories:
  Ex: Excellent
  Gd: Good
  TA: Typical - slight dampness allowed
  Fa: Fair - dampness or some cracking or settling
  Po: Poor - Severe cracking, settling, or wetness
  NA: No Basement
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  TA: 1309 (92.1%)
  Gd: 65 (4.6%)
  Fa: 45 (3.2%)
  Po: 2 (0.1%)

Test Value Distribution:
  TA: 1295 (91.6%)
  Fa: 59 (4.2%)
  Gd: 57 (4.0%)
  Po: 3 (0.2%)

Combined Value Distribution:
  TA: 2604 (91.9%)
  Gd: 122 (4.3%)
  Fa: 104 (3.7%)
  Po: 5 (0.2%)


In [42]:
# Fill missing BsmtCond with None
df_combined['BsmtCond'].fillna('None', inplace=True)
print('Missing values in BsmtCond after filling with None:', df_combined['BsmtCond'].isnull().sum())

Missing values in BsmtCond after filling with None: 0


#### BsmtQual

In [43]:
analyze_feature_missing_data('BsmtQual')


BsmtQual
------------------------------------------------------------
Total Missing: 81 values (2.8%)
  Train: 37 missing (2.5%)
  Test: 44 missing (3.0%)
  Combined: 81 missing (2.8%)
Feature: BsmtQual
Description: Evaluates the height of the basement
Type: Categorical

Categories:
  Ex: Excellent (100+ inches)
  Gd: Good (90-99 inches)
  TA: Typical (80-89 inches)
  Fa: Fair (70-79 inches)
  Po: Poor (<70 inches
  NA: No Basement
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  TA: 649 (45.7%)
  Gd: 618 (43.5%)
  Ex: 119 (8.4%)
  Fa: 35 (2.5%)

Test Value Distribution:
  TA: 634 (44.8%)
  Gd: 591 (41.8%)
  Ex: 137 (9.7%)
  Fa: 53 (3.7%)

Combined Value Distribution:
  TA: 1283 (45.2%)
  Gd: 1209 (42.6%)
  Ex: 256 (9.0%)
  Fa: 88 (3.1%)


In [44]:
# Fill missing BsmtQual with None
df_combined['BsmtQual'].fillna('None', inplace=True)
print('Missing values in BsmtQual after filling with None:', df_combined['BsmtQual'].isnull().sum())

Missing values in BsmtQual after filling with None: 0


#### BsmtFinType2

In [45]:
analyze_feature_missing_data('BsmtFinType2')


BsmtFinType2
------------------------------------------------------------
Total Missing: 80 values (2.7%)
  Train: 38 missing (2.6%)
  Test: 42 missing (2.9%)
  Combined: 80 missing (2.7%)
Feature: BsmtFinType2
Description: Rating of basement finished area (if multiple types)
Type: Categorical

Categories:
  GLQ: Good Living Quarters
  ALQ: Average Living Quarters
  BLQ: Below Average Living Quarters
  Rec: Average Rec Room
  LwQ: Low Quality
  Unf: Unfinshed
  NA: No Basement
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Unf: 1254 (88.3%)
  Rec: 54 (3.8%)
  LwQ: 46 (3.2%)
  BLQ: 33 (2.3%)
  ALQ: 19 (1.3%)
  GLQ: 14 (1.0%)

Test Value Distribution:
  Unf: 1237 (87.3%)
  Rec: 51 (3.6%)
  LwQ: 41 (2.9%)
  BLQ: 35 (2.5%)
  ALQ: 33 (2.3%)
  GLQ: 20 (1.4%)

Combined Value Distribution:
  Unf: 2491 (87.8%)
  Rec: 105 (3.7%)
  LwQ: 87 (3.1%)
  BLQ: 68 (2.4%)
  ALQ: 52 (1.8%)
  GLQ: 34 (1.2%)


In [46]:
# Fill missing BsmtFinType2 with None
df_combined['BsmtFinType2'].fillna('None', inplace=True)
print('Missing values in BsmtFinType2 after filling with None:', df_combined['BsmtFinType2'].isnull().sum())

Missing values in BsmtFinType2 after filling with None: 0


#### BsmtFinType1

In [47]:
analyze_feature_missing_data('BsmtFinType1')


BsmtFinType1
------------------------------------------------------------
Total Missing: 79 values (2.7%)
  Train: 37 missing (2.5%)
  Test: 42 missing (2.9%)
  Combined: 79 missing (2.7%)
Feature: BsmtFinType1
Description: Rating of basement finished area
Type: Categorical

Categories:
  GLQ: Good Living Quarters
  ALQ: Average Living Quarters
  BLQ: Below Average Living Quarters
  Rec: Average Rec Room
  LwQ: Low Quality
  Unf: Unfinshed
  NA: No Basement
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Unf: 430 (30.3%)
  GLQ: 416 (29.3%)
  ALQ: 220 (15.5%)
  BLQ: 148 (10.4%)
  Rec: 133 (9.4%)
  LwQ: 74 (5.2%)

Test Value Distribution:
  GLQ: 431 (30.4%)
  Unf: 421 (29.7%)
  ALQ: 209 (14.7%)
  Rec: 155 (10.9%)
  BLQ: 121 (8.5%)
  LwQ: 80 (5.6%)

Combined Value Distribution:
  Unf: 851 (30.0%)
  GLQ: 847 (29.8%)
  ALQ: 429 (15.1%)
  Rec: 288 (10.1%)
  BLQ: 269 (9.5%)
  LwQ: 154 (5.4%)


In [48]:
# Fill missing BsmtFinType1 with None
df_combined['BsmtFinType1'].fillna('None', inplace=True)
print('Missing values in BsmtFinType1 after filling with None:', df_combined['BsmtFinType1'].isnull().sum())

Missing values in BsmtFinType1 after filling with None: 0


#### MasVnrArea

In [49]:
analyze_feature_missing_data('MasVnrArea')


MasVnrArea
------------------------------------------------------------
Total Missing: 23 values (0.8%)
  Train: 8 missing (0.5%)
  Test: 15 missing (1.0%)
  Combined: 23 missing (0.8%)
Feature: MasVnrArea
Description: Masonry veneer area in square feet
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 0.0 – 1600.0
  Mean: 102.8, Median: 0.0
  Zero values: 861 (59.4%)

Test Stats:
  Range: 0.0 – 1290.0
  Mean: 100.7, Median: 0.0
  Zero values: 877 (60.7%)

Combined Stats:
  Range: 0.0 – 1600.0
  Mean: 101.7, Median: 0.0
  Zero values: 1738 (60.1%)


In [50]:
# Fill missing MasVnrArea with 0
df_combined['MasVnrArea'].fillna(0, inplace=True)
print('Missing values in MasVnrArea after filling with 0:', df_combined['MasVnrArea'].isnull().sum())

Missing values in MasVnrArea after filling with 0: 0


#### MSZoning

In [51]:
analyze_feature_missing_data('MSZoning')


MSZoning
------------------------------------------------------------
Total Missing: 4 values (0.1%)
  Train: 0 missing (0.0%)
  Test: 4 missing (0.3%)
  Combined: 4 missing (0.1%)
Feature: MSZoning
Description: Identifies the general zoning classification of the sale.
Type: Categorical

Categories:
  A: Agriculture
  C: Commercial
  FV: Floating Village Residential
  I: Industrial
  RH: Residential High Density
  RL: Residential Low Density
  RP: Residential Low Density Park
  RM: Residential Medium Density
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  RL: 1149 (78.8%)
  RM: 218 (15.0%)
  FV: 65 (4.5%)
  RH: 16 (1.1%)
  C (all): 10 (0.7%)

Test Value Distribution:
  RL: 1114 (76.6%)
  RM: 242 (16.6%)
  FV: 74 (5.1%)
  C (all): 15 (1.0%)
  RH: 10 (0.7%)

Combined Value Distribution:
  RL: 2263 (77.7%)
  RM: 460 (15.8%)
  FV: 139 (4.8%)
  RH: 26 (0.9%)
  C (all): 25 (0.9%)


In [52]:
# Compute mode of MSZoning per neighborhood from training data only
zoning_modes = df_train.groupby('Neighborhood')['MSZoning'].agg(lambda x: x.mode().iloc[0])

# Global fallback mode (from training only)
global_mode = df_train['MSZoning'].mode().iloc[0]

# Fill missing values using training modes, fallback to global mode if Neighborhood not in training
df_combined['MSZoning'] = df_combined.groupby('Neighborhood')['MSZoning'].transform(
    lambda x: x.fillna(zoning_modes.get(x.name, global_mode))
)

missing_count = df_combined['MSZoning'].isnull().sum()
print(f"Missing values in MSZoning after filling with training-based modes: {missing_count}")


Missing values in MSZoning after filling with training-based modes: 0


#### BsmtFullBath

In [53]:
analyze_feature_missing_data('BsmtFullBath')


BsmtFullBath
------------------------------------------------------------
Total Missing: 2 values (0.1%)
  Train: 0 missing (0.0%)
  Test: 2 missing (0.1%)
  Combined: 2 missing (0.1%)
Feature: BsmtFullBath
Description: Basement full bathrooms
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 0.0 – 3.0
  Mean: 0.4, Median: 0.0
  Zero values: 856 (58.7%)

Test Stats:
  Range: 0.0 – 3.0
  Mean: 0.4, Median: 0.0
  Zero values: 849 (58.3%)

Combined Stats:
  Range: 0.0 – 3.0
  Mean: 0.4, Median: 0.0
  Zero values: 1705 (58.5%)


In [54]:
# Fill missing BsmtFullBath with 0
df_combined['BsmtFullBath'].fillna(0, inplace=True)
print('Missing values in BsmtFullBath after filling with 0:', df_combined['BsmtFullBath'].isnull().sum())

Missing values in BsmtFullBath after filling with 0: 0


#### BsmtHalfBath

In [55]:
analyze_feature_missing_data('BsmtHalfBath')


BsmtHalfBath
------------------------------------------------------------
Total Missing: 2 values (0.1%)
  Train: 0 missing (0.0%)
  Test: 2 missing (0.1%)
  Combined: 2 missing (0.1%)
Feature: BsmtHalfBath
Description: Basement half bathrooms
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 0.0 – 2.0
  Mean: 0.1, Median: 0.0
  Zero values: 1376 (94.4%)

Test Stats:
  Range: 0.0 – 2.0
  Mean: 0.1, Median: 0.0
  Zero values: 1364 (93.6%)

Combined Stats:
  Range: 0.0 – 2.0
  Mean: 0.1, Median: 0.0
  Zero values: 2740 (94.0%)


In [56]:
# Fill missing BsmtHalfBath with 0
df_combined['BsmtHalfBath'].fillna(0, inplace=True)
print('Missing values in BsmtHalfBath after filling with 0:', df_combined['BsmtHalfBath'].isnull().sum())

Missing values in BsmtHalfBath after filling with 0: 0


#### Functional

In [57]:
analyze_feature_missing_data('Functional')


Functional
------------------------------------------------------------
Total Missing: 2 values (0.1%)
  Train: 0 missing (0.0%)
  Test: 2 missing (0.1%)
  Combined: 2 missing (0.1%)
Feature: Functional
Description: Home functionality (Assume typical unless deductions are warranted)
Type: Categorical

Categories:
  Typ: Typical Functionality
  Min1: Minor Deductions 1
  Min2: Minor Deductions 2
  Mod: Moderate Deductions
  Maj1: Major Deductions 1
  Maj2: Major Deductions 2
  Sev: Severely Damaged
  Sal: Salvage only
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  Typ: 1358 (93.1%)
  Min2: 34 (2.3%)
  Min1: 31 (2.1%)
  Mod: 15 (1.0%)
  Maj1: 14 (1.0%)
  Maj2: 5 (0.3%)
  Sev: 1 (0.1%)

Test Value Distribution:
  Typ: 1357 (93.1%)
  Min2: 36 (2.5%)
  Min1: 34 (2.3%)
  Mod: 20 (1.4%)
  Maj1: 5 (0.3%)
  Maj2: 4 (0.3%)
  Sev: 1 (0.1%)

Combined Value Distribution:
  Typ: 2715 (93.1%)
  Min2: 70 (2.4%)
  Min1: 65 (2.2%)
  Mod

In [58]:
# Fill missing Functional with the mode from the training set, Typ
df_combined['Functional'].fillna('Typ', inplace=True)
print('Missing values in Functional after filling with Typ:', df_combined['Functional'].isnull().sum())

Missing values in Functional after filling with Typ: 0


#### Utilities

In [59]:
analyze_feature_missing_data('Utilities')


Utilities
------------------------------------------------------------
Total Missing: 2 values (0.1%)
  Train: 0 missing (0.0%)
  Test: 2 missing (0.1%)
  Combined: 2 missing (0.1%)
Feature: Utilities
Description: Type of utilities available
Type: Categorical

Categories:
  AllPub: All public Utilities (E,G,W,& S)
  NoSewr: Electricity, Gas, and Water (Septic Tank)
  NoSeWa: Electricity and Gas Only
  ELO: Electricity only
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  AllPub: 1457 (99.9%)
  NoSeWa: 1 (0.1%)

Test Value Distribution:
  AllPub: 1457 (100.0%)

Combined Value Distribution:
  AllPub: 2914 (100.0%)
  NoSeWa: 1 (0.0%)


In [60]:
# Fill missing Utilities with the mode from the training set, AllPub
df_combined['Utilities'].fillna('AllPub', inplace=True)
print('Missing values in Utilities after filling with AllPub:', df_combined['Utilities'].isnull().sum())

Missing values in Utilities after filling with AllPub: 0


#### Electrical

In [61]:
analyze_feature_missing_data('Electrical')


Electrical
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 1 missing (0.1%)
  Test: 0 missing (0.0%)
  Combined: 1 missing (0.0%)
Feature: Electrical
Description: Electrical system
Type: Categorical

Categories:
  SBrkr: Standard Circuit Breakers & Romex
  FuseA: Fuse Box over 60 AMP and all Romex wiring (Average)
  FuseF: 60 AMP Fuse Box and mostly Romex wiring (Fair)
  FuseP: 60 AMP Fuse Box and mostly knob & tube wiring (poor)
  Mix: Mixed
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  SBrkr: 1332 (91.4%)
  FuseA: 94 (6.5%)
  FuseF: 27 (1.9%)
  FuseP: 3 (0.2%)
  Mix: 1 (0.1%)

Test Value Distribution:
  SBrkr: 1337 (91.6%)
  FuseA: 94 (6.4%)
  FuseF: 23 (1.6%)
  FuseP: 5 (0.3%)

Combined Value Distribution:
  SBrkr: 2669 (91.5%)
  FuseA: 188 (6.4%)
  FuseF: 50 (1.7%)
  FuseP: 8 (0.3%)
  Mix: 1 (0.0%)


In [62]:
# Fill missing Electrical with with the mode from the training set, SBrkr
df_combined['Electrical'].fillna('SBrkr', inplace=True)
print('Missing values in Electrical after filling with SBrkr:', df_combined['Electrical'].isnull().sum())

Missing values in Electrical after filling with SBrkr: 0


#### KitchenQual

In [63]:
analyze_feature_missing_data('KitchenQual')


KitchenQual
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: KitchenQual
Description: Kitchen quality
Type: Categorical

Categories:
  Ex: Excellent
  Gd: Good
  TA: Typical/Average
  Fa: Fair
  Po: Poor
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  TA: 735 (50.4%)
  Gd: 586 (40.2%)
  Ex: 98 (6.7%)
  Fa: 39 (2.7%)

Test Value Distribution:
  TA: 757 (51.9%)
  Gd: 565 (38.8%)
  Ex: 105 (7.2%)
  Fa: 31 (2.1%)

Combined Value Distribution:
  TA: 1492 (51.2%)
  Gd: 1151 (39.5%)
  Ex: 203 (7.0%)
  Fa: 70 (2.4%)


In [64]:
# Fill missing KitchenQual with with the mode from the training set, TA
df_combined['KitchenQual'].fillna('TA', inplace=True)
print('Missing values in KitchenQual after filling with TA:', df_combined['KitchenQual'].isnull().sum())


Missing values in KitchenQual after filling with TA: 0


#### TotalBsmtSF

In [65]:
analyze_feature_missing_data('TotalBsmtSF')


TotalBsmtSF
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: TotalBsmtSF
Description: Total square feet of basement area
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 0.0 – 3206.0
  Mean: 1052.5, Median: 991.0
  Zero values: 37 (2.5%)

Test Stats:
  Range: 0.0 – 5095.0
  Mean: 1046.1, Median: 988.0
  Zero values: 41 (2.8%)

Combined Stats:
  Range: 0.0 – 5095.0
  Mean: 1049.3, Median: 988.5
  Zero values: 78 (2.7%)


In [66]:
# Fill missing TotalBsmtSF with 0
df_combined['TotalBsmtSF'].fillna(0, inplace=True)
print('Missing values in TotalBsmtSF after filling with 0:', df_combined['TotalBsmtSF'].isnull().sum())


Missing values in TotalBsmtSF after filling with 0: 0


#### BsmtUnfSF

In [67]:
analyze_feature_missing_data('BsmtUnfSF')


BsmtUnfSF
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: BsmtUnfSF
Description: Unfinished square feet of basement area
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 0.0 – 2336.0
  Mean: 567.1, Median: 477.5
  Zero values: 118 (8.1%)

Test Stats:
  Range: 0.0 – 2140.0
  Mean: 554.3, Median: 460.0
  Zero values: 123 (8.4%)

Combined Stats:
  Range: 0.0 – 2336.0
  Mean: 560.7, Median: 467.0
  Zero values: 241 (8.3%)


In [68]:
# Fill missing BsmtUnfSF with 0
df_combined['BsmtUnfSF'].fillna(0, inplace=True)
print('Missing values in BsmtUnfSF after filling with 0:', df_combined['BsmtUnfSF'].isnull().sum())


Missing values in BsmtUnfSF after filling with 0: 0


#### BsmtFinSF2

In [69]:
analyze_feature_missing_data('BsmtFinSF2')


BsmtFinSF2
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: BsmtFinSF2
Description: Type 2 finished square feet
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 0.0 – 1474.0
  Mean: 46.6, Median: 0.0
  Zero values: 1291 (88.5%)

Test Stats:
  Range: 0.0 – 1526.0
  Mean: 52.6, Median: 0.0
  Zero values: 1278 (87.7%)

Combined Stats:
  Range: 0.0 – 1526.0
  Mean: 49.6, Median: 0.0
  Zero values: 2569 (88.1%)


In [70]:
# Fill missing BsmtFinSF2 with 0
df_combined['BsmtFinSF2'].fillna(0, inplace=True)
print('Missing values in BsmtFinSF2 after filling with 0:', df_combined['BsmtFinSF2'].isnull().sum())


Missing values in BsmtFinSF2 after filling with 0: 0


#### BsmtFinSF1

In [71]:
analyze_feature_missing_data('BsmtFinSF1')


BsmtFinSF1
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: BsmtFinSF1
Description: Type 1 finished square feet
Type: Numerical
------------------------------------------------------------

Data type: Numerical

Train Stats:
  Range: 0.0 – 2188.0
  Mean: 438.8, Median: 382.0
  Zero values: 467 (32.0%)

Test Stats:
  Range: 0.0 – 4010.0
  Mean: 439.2, Median: 350.5
  Zero values: 462 (31.7%)

Combined Stats:
  Range: 0.0 – 4010.0
  Mean: 439.0, Median: 368.0
  Zero values: 929 (31.9%)


In [72]:
# Fill missing BsmtFinSF1 with 0
df_combined['BsmtFinSF1'].fillna(0, inplace=True)
print('Missing values in BsmtFinSF1 after filling with 0:', df_combined['BsmtFinSF1'].isnull().sum())


Missing values in BsmtFinSF1 after filling with 0: 0


#### Exterior2nd

In [73]:
analyze_feature_missing_data('Exterior2nd')


Exterior2nd
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: Exterior2nd
Description: Exterior covering on house (if more than one material)
Type: Categorical

Categories:
  AsbShng: Asbestos Shingles
  AsphShn: Asphalt Shingles
  BrkComm: Brick Common
  BrkFace: Brick Face
  CBlock: Cinder Block
  CemntBd: Cement Board
  HdBoard: Hard Board
  ImStucc: Imitation Stucco
  MetalSd: Metal Siding
  Other: Other
  Plywood: Plywood
  PreCast: PreCast
  Stone: Stone
  Stucco: Stucco
  VinylSd: Vinyl Siding
  Wd Sdng: Wood Siding
  WdShing: Wood Shingles
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  VinylSd: 504 (34.6%)
  MetalSd: 214 (14.7%)
  HdBoard: 207 (14.2%)
  Wd Sdng: 197 (13.5%)
  Plywood: 142 (9.7%)
  CmentBd: 59 (4.0%)
  Wd Shng: 38 (2.6%)
  BrkFace: 25 (1.7%)
  Stucco: 25 (1.7%)
  Asb

In [74]:
# Replace typos in Exterior2nd

# Strip whitespaces 
df_combined['Exterior2nd'] = df_combined['Exterior2nd'].str.strip()

replace_map = {
    'BrkComm': 'Brk Cmn',
    'CemntBd': 'CmentBd',
    'WdShing': 'Wd Shng'
}

df_combined['Exterior2nd'] = df_combined['Exterior2nd'].replace(replace_map)


In [75]:
# Fill missing Exterior2nd with the mode from the training set, VinylSd
df_combined['Exterior2nd'].fillna('VinylSd', inplace=True)
print('Missing values in Exterior2nd after filling with VinylSd:', df_combined['Exterior2nd'].isnull().sum())


Missing values in Exterior2nd after filling with VinylSd: 0


#### Exterior1st

In [76]:
analyze_feature_missing_data('Exterior1st')


Exterior1st
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: Exterior1st
Description: Exterior covering on house
Type: Categorical

Categories:
  AsbShng: Asbestos Shingles
  AsphShn: Asphalt Shingles
  BrkComm: Brick Common
  BrkFace: Brick Face
  CBlock: Cinder Block
  CemntBd: Cement Board
  HdBoard: Hard Board
  ImStucc: Imitation Stucco
  MetalSd: Metal Siding
  Other: Other
  Plywood: Plywood
  PreCast: PreCast
  Stone: Stone
  Stucco: Stucco
  VinylSd: Vinyl Siding
  Wd Sdng: Wood Siding
  WdShing: Wood Shingles
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  VinylSd: 515 (35.3%)
  HdBoard: 222 (15.2%)
  MetalSd: 220 (15.1%)
  Wd Sdng: 206 (14.1%)
  Plywood: 108 (7.4%)
  CemntBd: 60 (4.1%)
  BrkFace: 50 (3.4%)
  WdShing: 26 (1.8%)
  Stucco: 24 (1.6%)
  AsbShng: 20 (1.4%)
  BrkComm: 2

In [77]:
# Replace the same typos in Exterior1st

# Strip whitespaces 
df_combined['Exterior1st'] = df_combined['Exterior1st'].str.strip()


df_combined['Exterior1st'] = df_combined['Exterior1st'].replace(replace_map)

In [78]:
# Fill missing Exterior1st with the mode from the training set, VinylSd
df_combined['Exterior1st'].fillna('VinylSd', inplace=True)
print('Missing values in Exterior1st after filling with VinylSd:', df_combined['Exterior1st'].isnull().sum())

Missing values in Exterior1st after filling with VinylSd: 0


#### SaleType

In [79]:
analyze_feature_missing_data('SaleType')


SaleType
------------------------------------------------------------
Total Missing: 1 values (0.0%)
  Train: 0 missing (0.0%)
  Test: 1 missing (0.1%)
  Combined: 1 missing (0.0%)
Feature: SaleType
Description: Type of sale
Type: Categorical

Categories:
  WD: Warranty Deed - Conventional
  CWD: Warranty Deed - Cash
  VWD: Warranty Deed - VA Loan
  New: Home just constructed and sold
  COD: Court Officer Deed/Estate
  Con: Contract 15% Down payment regular terms
  ConLw: Contract Low Down payment and low interest
  ConLI: Contract Low Interest
  ConLD: Contract Low Down
  Oth: Other
------------------------------------------------------------

Data type: Categorical

Train Value Distribution:
  WD: 1267 (86.9%)
  New: 120 (8.2%)
  COD: 43 (2.9%)
  ConLD: 9 (0.6%)
  ConLI: 5 (0.3%)
  ConLw: 5 (0.3%)
  CWD: 4 (0.3%)
  Oth: 3 (0.2%)
  Con: 2 (0.1%)

Test Value Distribution:
  WD: 1258 (86.3%)
  New: 117 (8.0%)
  COD: 44 (3.0%)
  ConLD: 17 (1.2%)
  CWD: 8 (0.5%)
  Oth: 4 (0.3%)
  ConLI: 

In [80]:
# Fill missing SaleType with the mode from the training set, WD
df_combined['SaleType'].fillna('WD', inplace=True)
print('Missing values in SaleType after filling with WD:', df_combined['SaleType'].isnull().sum())

Missing values in SaleType after filling with WD: 0


### 4.3 Missing Data Validation

Verify that the systematic treatment approach has eliminated all missing data across all features.

In [81]:
all_missing = df_combined.drop('dataset_source', axis=1).isnull().sum()
remaining_missing = all_missing[all_missing > 0].sort_values(ascending=False)

print(f"Missing data summary:")
print(f"Total features: {len(all_missing)}")
print(f"Features with missing values: {len(remaining_missing)}")
print(f"Total missing values: {remaining_missing.sum()}")

if len(remaining_missing) == 0:
    print("No missing values remaining in dataset")
else:
    print(f"\nRemaining missing values:")
    for feature, count in remaining_missing.items():
        pct = (count / len(df_combined)) * 100
        print(f"  {feature}: {count} ({pct:.2f}%)")

Missing data summary:
Total features: 80
Features with missing values: 0
Total missing values: 0
No missing values remaining in dataset


Final validation confirms systematic approach has successfully eliminated all missing values.

## 5. Data Quality Corrections

Address data quality issues identified through systematic validation by loading and analyzing logged quality issues to determine appropriate corrections.

### 5.1 Import Data Quality Issues

Load data quality issues


In [82]:
# Load the data quality issues
df_issues = pd.read_csv("../data/logs/data_quality_issues.csv")

# First, sort by IssueCount, then by Issue
df_issues_sorted = df_issues.sort_values(by=["IssueCount", "Issue"], ascending=[False, True])

# Display
pd.set_option('display.max_rows', None)
display(df_issues_sorted.style.hide(axis="index"))


Id,Issue,IssueCount
2550,Construction year is after the house was sold,4
2550,Garage built after the house was sold,4
2550,MiscVal > 0 but MiscFeature is missing,4
2550,Remodel date is after the house was sold,4
2218,Basement has area but BsmtQual is missing,1
2219,Basement has area but BsmtQual is missing,1
2593,Garage built after the house was sold,1
30,Garage built before the house,1
94,Garage built before the house,1
325,Garage built before the house,1


### 5.2 House with ID 2250

Look at the different features of the house with ID 2250 to understand the nature of the data quality issues.

In [83]:
display(
    df_combined[df_combined['Id'] == 2550].T.style.hide(axis="columns")
)

Id,2550
MSSubClass,20
MSZoning,RL
LotFrontage,128.000000
LotArea,39290
Street,Pave
Alley,None
LotShape,IR1
LandContour,Bnk
Utilities,AllPub
LotConfig,Inside


In [84]:
# Set MiscFeature to 'Othr'
df_combined.loc[df_combined['Id'] == 2550, 'MiscFeature'] = 'Othr'

# Display the house with ID 2550 after fixing the MiscFeature
display(
    df_combined[df_combined['Id'] == 2550][['Id', 'MiscVal', 'MiscFeature']].style.hide(axis="index")
)

# Remove the house with ID 2550 from the issues list
df_issues_sorted = df_issues_sorted[df_issues_sorted['Id'] != 2550]

Id,MiscVal,MiscFeature
2550,17000,Othr


**Conclusion for Property ID 2550**
- Four data issues were flagged, including future YearBuilt, YearRemodAdd, and GarageYrBlt, and a missing MiscFeature despite a high MiscVal.
- Given SaleType = New and SaleCondition = Partial, these values are plausible and reflect a home not fully completed at the time of sale.
- Fix applied: MiscFeature was set to 'Othr'; other issues were accepted as contextually valid.

### 5.3 Analyze data quality issues

##### Function to Analyze Data Quality Issues

In [85]:
def show_issue_details(issue_name: str, columns: list):
    """
    Displays selected columns for all houses affected by a specific data quality issue.

    Parameters:
    - issue_name (str): The name of the issue as logged in df_issues_sorted['Issue']
    - columns (list): List of column names to display
    """
    # Get IDs for houses with the given issue
    issue_ids = df_issues_sorted[df_issues_sorted['Issue'] == issue_name]['Id'].unique()

    # Filter original dataset for those IDs and columns
    df_display = df_combined[df_combined['Id'].isin(issue_ids)][columns]

    # Show results
    print(f"Issue: {issue_name} — {len(issue_ids)} house(s) affected")
    display(df_display.style.hide(axis="index"))


#### Basement has area but BsmtQual is missing

In [86]:
# Columns to show for basement-related issues
cols_to_show_basement = [
    'Id',
    'TotalBsmtSF',     
    'BsmtFinSF1',      
    'BsmtFinSF2',
    'BsmtUnfSF',       
    'BsmtExposure',    
    'BsmtFinType1',    
    'BsmtFinType2',
    'BsmtCond',        
    'BsmtQual',        
    'YearBuilt',
    'YearRemodAdd',
    'YrSold',
    'MoSold',
    'OverallQual',
    'Functional',
    'LowQualFinSF'
]


show_issue_details("Basement has area but BsmtQual is missing", cols_to_show_basement)


Issue: Basement has area but BsmtQual is missing — 2 house(s) affected


Id,TotalBsmtSF,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtCond,BsmtQual,YearBuilt,YearRemodAdd,YrSold,MoSold,OverallQual,Functional,LowQualFinSF
2218,173.000000,0.000000,0.000000,173.000000,No,Unf,Unf,Fa,None,1895,1950,2008,7,4,Typ,0
2219,356.000000,0.000000,0.000000,356.000000,No,Unf,Unf,TA,None,1910,2000,2008,5,4,Typ,0


**Conclusion for Basement has area but BsmtQual is missing**
To properties have missing BsmtQual, but both basements are fully unfinished. We leave BsmtQual as None, as there is no meaningful quality grading for unfinished basements.

#### Garage built after the house was sold

In [87]:
cols_to_show_garage = [
    'Id',
    'YearBuilt',
    'YrSold',
    'MoSold',
    'GarageYrBlt',
    'GarageType',
    'GarageFinish',
    'GarageCars',
    'GarageArea',
    'SaleType',
    'SaleCondition',
]

show_issue_details("Garage built after the house was sold", cols_to_show_garage)

# Remove the houses from the issues list
df_issues_sorted = df_issues_sorted[df_issues_sorted['Issue'] != "Garage built after the house was sold"]


Issue: Garage built after the house was sold — 1 house(s) affected


Id,YearBuilt,YrSold,MoSold,GarageYrBlt,GarageType,GarageFinish,GarageCars,GarageArea,SaleType,SaleCondition
2593,2006,2007,9,2007.000000,Attchd,RFn,2.000000,502.000000,New,Partial



**Conclusion for Garage built after the house was sold**
The garage year (GarageYrBlt) is 2007 (corrected from 2207) the same year the house was sold. Since the SaleType is New and the SaleCondition is Partial, this suggests the property was not fully completed when assessed. The GarageFinish is RFn (Rough Finished), which further supports that the garage was in the process of completion. No correction is needed.

#### Garage built before the house

In [88]:
cols_to_show_garage_before = [
    'Id',
    'GarageYrBlt',
    'YearBuilt',
    'YrSold',
    'MoSold',
    'YearRemodAdd',
    'GarageType',
    'GarageFinish',
    'GarageQual',
    'GarageCond',
    'GarageCars',
    'GarageArea',
    'SaleType',
    'SaleCondition',
]

show_issue_details("Garage built before the house", cols_to_show_garage_before)

# Remove the houses from the issues list
df_issues_sorted = df_issues_sorted[df_issues_sorted['Issue'] != "Garage built before the house"]


Issue: Garage built before the house — 18 house(s) affected


Id,GarageYrBlt,YearBuilt,YrSold,MoSold,YearRemodAdd,GarageType,GarageFinish,GarageQual,GarageCond,GarageCars,GarageArea,SaleType,SaleCondition
30,1920.000000,1927,2008,5,1950,Detchd,Unf,Fa,TA,1.000000,240.000000,WD,Normal
94,1900.000000,1910,2007,11,1998,Detchd,Unf,TA,TA,2.000000,506.000000,WD,Normal
325,1961.000000,1967,2010,6,2007,BuiltIn,Fin,TA,TA,2.000000,564.000000,WD,Normal
601,2003.000000,2005,2006,6,2005,BuiltIn,Fin,TA,TA,2.000000,736.000000,WD,Normal
737,1949.000000,1950,2006,7,1950,Detchd,Unf,TA,TA,2.000000,400.000000,WD,Normal
1104,1954.000000,1959,2006,7,1959,BuiltIn,Fin,TA,TA,2.000000,539.000000,WD,Normal
1377,1925.000000,1930,2008,4,1950,Detchd,Unf,Fa,TA,1.000000,160.000000,WD,Normal
1415,1922.000000,1923,2008,6,2000,Detchd,Unf,TA,TA,2.000000,370.000000,WD,Normal
1419,1962.000000,1963,2008,8,1963,Detchd,Unf,TA,TA,1.000000,336.000000,COD,Normal
1522,1956.000000,1959,2010,6,1959,Attchd,Unf,TA,TA,2.000000,440.000000,WD,Normal


**Conclusion for Garage Built Before the House**
- Most cases involve detached garages (GarageType = Detchd), which makes it plausible that the garage existed on the lot before the current house was built or rebuilt.
- The year differences are typically small, often within a margin of 1–10 years, which is common in older properties or due to approximate registration.
- No unusual sale conditions were found; most are marked as WD/Normal, suggesting standard market sales.
- Some garages are marked as BuiltIn or Attchd, but still show only a one-year difference, likely due to recording discrepancies.

Given these points, the data appears plausible and historically consistent.

#### No kitchen above grade

In [89]:
cols_to_show_kitchen = [
    'Id',
    'KitchenAbvGr',
    'KitchenQual',
    'TotRmsAbvGrd',
    'GrLivArea',
    'LowQualFinSF',
    'OverallQual',
    'HouseStyle',
    'YearBuilt',
    'YearRemodAdd',
    'Functional',
    'SaleCondition',
    'BldgType',
    'SaleType',
    'SaleCondition',
]

show_issue_details("No kitchen above grade", cols_to_show_kitchen)

# Remove the houses from the issues list
df_issues_sorted = df_issues_sorted[df_issues_sorted['Issue'] != "No kitchen above grade"]


Issue: No kitchen above grade — 3 house(s) affected


Id,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,GrLivArea,LowQualFinSF,OverallQual,HouseStyle,YearBuilt,YearRemodAdd,Functional,SaleCondition,BldgType,SaleType,SaleCondition
955,0,TA,4,980,0,6,SFoyer,1975,1975,Typ,AdjLand,Duplex,WD,AdjLand
2588,0,TA,4,918,0,7,SLvl,1975,1975,Typ,Normal,1Fam,WD,Normal
2860,0,TA,4,1012,0,6,SFoyer,1975,1975,Typ,AdjLand,Duplex,WD,AdjLand


In [90]:
# Set KitchenAbvGr to 1 for houses with no kitchen above grade
df_combined.loc[df_combined['KitchenAbvGr'] == 0, 'KitchenAbvGr'] = 1

# Display the houses with no kitchen above grade after fixing
display(
    df_combined[df_combined['Id'].isin([955, 2588, 2860])][['Id', 'KitchenAbvGr', 'KitchenQual']].style.hide(axis="index")

)

Id,KitchenAbvGr,KitchenQual
955,1,TA
2588,1,TA
2860,1,TA


 **Conclusion for Houses with No Kitchen Above Grade**

  - All 3 affected houses (IDs: 955, 2588, 2860) have KitchenAbvGr = 0 but show KitchenQual = TA, indicating kitchen quality is recorded despite zero
  kitchen count.
  - The houses have substantial living areas (918-1012 sqft) and 4 rooms above grade, making it implausible they lack kitchens entirely.
  - All properties are from 1975, suggesting a systematic data entry error or coding issue from the original source.
  - House styles include SFoyer (Split Foyer) and SLvl (Split Level), which are multi-level designs that typically have kitchens on the main level
  above grade.
  - Building types include both Duplex and 1Fam (Single Family), showing the error spans multiple property configurations.
  - Two houses have AdjLand sale conditions, but one has Normal sale condition, indicating the issue is not related to special sale circumstances.

  Given these points, the zero kitchen count appears to be a clear data entry error inconsistent with the house styles and building types, and is therefore set to 1 for all three houses, as every functional residence must have at least one kitchen.

#### Pool has area but PoolQC is missing

In [91]:
cols_to_show_pool = [
    'Id',
    'PoolArea',
    'PoolQC',
    'OverallQual',
    'YearBuilt',
    'MoSold', 'YrSold',
    'YearRemodAdd',
    'Functional',
    'ExterQual',
    'MiscFeature', 'MiscVal',
    'SaleType', 'SaleCondition',
]
show_issue_details("Pool has area but PoolQC is missing", cols_to_show_pool)

# Remove the houses from the issues list
df_issues_sorted = df_issues_sorted[df_issues_sorted['Issue'] != "Pool has area but PoolQC is missing"]

Issue: Pool has area but PoolQC is missing — 3 house(s) affected


Id,PoolArea,PoolQC,OverallQual,YearBuilt,MoSold,YrSold,YearRemodAdd,Functional,ExterQual,MiscFeature,MiscVal,SaleType,SaleCondition
2421,368,None,4,1953,2,2007,1953,Min1,TA,None,0,WD,Normal
2504,444,None,6,1984,4,2007,1984,Min2,TA,None,0,WD,Normal
2600,561,None,3,1953,6,2007,1953,Min1,TA,None,0,WD,Normal


#### Show houses with PoolQC


In [92]:
# Houses with PoolQC
df_pools = df_combined[df_combined['PoolQC'] != "None"][cols_to_show_pool]

display(df_pools.style.hide(axis="index"))

Id,PoolArea,PoolQC,OverallQual,YearBuilt,MoSold,YrSold,YearRemodAdd,Functional,ExterQual,MiscFeature,MiscVal,SaleType,SaleCondition
198,512,Ex,8,1918,3,2006,1990,Typ,Gd,None,0,WD,Abnorml
811,648,Fa,6,1974,1,2006,1999,Typ,TA,None,0,WD,Normal
1171,576,Gd,6,1977,7,2008,1977,Typ,TA,None,0,WD,Normal
1183,555,Ex,10,1996,7,2007,1996,Typ,Gd,None,0,WD,Abnorml
1387,519,Fa,7,1978,7,2006,1978,Typ,TA,TenC,2000,WD,Normal
1424,738,Gd,6,1966,8,2006,1966,Typ,Gd,None,0,WD,Alloca
1975,144,Ex,10,2003,2,2008,2003,Typ,Ex,None,0,WD,Normal
2574,228,Ex,8,1986,8,2007,1986,Typ,Gd,None,0,WD,Normal
2711,800,Gd,7,1974,1,2006,1974,Mod,Gd,None,0,WD,Normal


In [93]:
# Set PoolArea to 0 for houses with PoolQC missing
df_combined.loc[df_combined['PoolQC'] == "None", 'PoolArea'] = 0

# Display the houses with PoolQC missing after fixing
display(
    df_combined[df_combined['Id'].isin([2421, 2504, 2600])][['Id', 'PoolArea', 'PoolQC']].style.hide(axis="index")
)

Id,PoolArea,PoolQC
2421,0,None
2504,0,None
2600,0,None


**Conclusion for Pool has area but PoolQC is missing**

  Analysis of the 9 confirmed pools reveals a clear pattern: all houses with pools have OverallQual ≥ 6 (range: 6-10, even if we only look at the training data), indicating above-average to excellent property quality. In contrast, two of the three problematic houses (IDs 2421 and 2600) have OverallQual ≤ 4, representing below-average to fair quality properties.

  Given that:
  - All confirmed pools are in higher-quality homes (OverallQual ≥ 6)
  - PoolQC = "None" explicitly means "No Pool" per data documentation
  - Houses with OverallQual 3-4 are inconsistent with the established pool ownership pattern

  
  Decision: Set PoolArea = 0 for all three houses (IDs 2421, 2504, 2600) to maintain data consistency and respect the explicit "No Pool" designation in PoolQC.


#### Remodel date before construction year

In [94]:
cols_to_show_remodel = [
      'Id',
      'YearBuilt', 'YearRemodAdd', 'YrSold',
      'OverallQual', 'OverallCond',
      'SaleType', 'SaleCondition',
      'HouseStyle', 'BldgType',
      'Foundation', 'Exterior1st',
  ]

show_issue_details("Remodel date before construction year", cols_to_show_remodel)

# Remove the houses from the issues list
df_issues_sorted = df_issues_sorted[df_issues_sorted['Issue'] != "Remodel date before construction year"]

Issue: Remodel date before construction year — 1 house(s) affected


Id,YearBuilt,YearRemodAdd,YrSold,OverallQual,OverallCond,SaleType,SaleCondition,HouseStyle,BldgType,Foundation,Exterior1st
1877,2002,2001,2009,7,5,WD,Normal,1Story,1Fam,PConc,VinylSd


In [95]:
# Fix remodel date before construction year
df_combined.loc[df_combined['Id'] == 1877, 'YearRemodAdd'] = 2002

# Display the house with ID 1877 after fixing
display(
    df_combined[df_combined['Id'] == 1877][['Id', 'YearBuilt', 'YearRemodAdd']].style.hide(axis="index")
)

Id,YearBuilt,YearRemodAdd
1877,2002,2002


**Conclusion for Remodel Date Before Construction Year**

House ID 1877 shows YearRemodAdd (2001) preceding YearBuilt (2002), which is logically impossible. This represents a clear data entry error where
the remodel date was recorded one year before the house was actually constructed.

Decision: Set YearRemodAdd = YearBuilt = 2002 for ID 1877. According to the data documentation, when no remodeling or additions have occurred,
YearRemodAdd should equal the construction date. 


#### Remodel date is after the house was sold

In [96]:
cols_to_show_remodel_after_sale = [
      'Id',
      'YearBuilt', 'YearRemodAdd', 'YrSold', 'MoSold',
      'SaleType', 'SaleCondition',
      'OverallQual', 'OverallCond',
      'HouseStyle', 'BldgType',
      'Functional'
  ]

show_issue_details("Remodel date is after the house was sold", cols_to_show_remodel_after_sale)

# Remove the houses from the issues list
df_issues_sorted = df_issues_sorted[df_issues_sorted['Issue'] != "Remodel date is after the house was sold"]

Issue: Remodel date is after the house was sold — 2 house(s) affected


Id,YearBuilt,YearRemodAdd,YrSold,MoSold,SaleType,SaleCondition,OverallQual,OverallCond,HouseStyle,BldgType,Functional
2296,2007,2008,2007,6,New,Partial,8,5,2Story,1Fam,Typ


**Conclusion for Remodel Date After House Was Sold**

  House ID 2296 shows YearRemodAdd (2008) after YrSold (2007), but this is actually logical given the sale context. The property was sold as SaleType = "New" with SaleCondition = "Partial", indicating a newly constructed home that was not completed when sold.

#### Basement has area but BsmtQual is missing

In [97]:
cols_to_show_basement = [
      'Id',
      'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2',
      'BsmtQual', 'BsmtCond', 'BsmtExposure',
      'BsmtFinType1', 'BsmtFinType2',
      'BsmtFullBath', 'BsmtHalfBath',
      'LowQualFinSF', 'Functional',
      'Foundation', 'OverallQual', 'OverallCond',
      'YearBuilt', 'YearRemodAdd',
      'HouseStyle', 'BldgType',
      '1stFlrSF', 'GrLivArea',
      'SaleType', 'SaleCondition'
  ]

show_issue_details("Basement has area but BsmtQual is missing", cols_to_show_basement)

# Remove the houses from the issues list
df_issues_sorted = df_issues_sorted[df_issues_sorted['Issue'] != "Basement has area but BsmtQual is missing"]

Issue: Basement has area but BsmtQual is missing — 2 house(s) affected


Id,TotalBsmtSF,BsmtUnfSF,BsmtFinSF1,BsmtFinSF2,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,LowQualFinSF,Functional,Foundation,OverallQual,OverallCond,YearBuilt,YearRemodAdd,HouseStyle,BldgType,1stFlrSF,GrLivArea,SaleType,SaleCondition
2218,173.000000,173.000000,0.000000,0.000000,None,Fa,No,Unf,Unf,0.000000,0.000000,0,Typ,Stone,4,7,1895,1950,2Story,1Fam,825,1361,WD,Normal
2219,356.000000,356.000000,0.000000,0.000000,None,TA,No,Unf,Unf,0.000000,0.000000,0,Typ,PConc,4,7,1910,2000,1.5Fin,1Fam,671,1049,WD,Normal


In [98]:
 # Fix basement quality missing values  
df_combined.loc[df_combined['Id'] == 2218, 'BsmtQual'] = 'Po'                                                                  
df_combined.loc[df_combined['Id'] == 2219, 'BsmtQual'] = 'Fa' 

# Display the houses with missing BsmtQual after fixing
display(
    df_combined[df_combined['Id'].isin([2218, 2219])][['Id', 'BsmtQual', 'TotalBsmtSF']].style.hide(axis="index")
)

Id,BsmtQual,TotalBsmtSF
2218,Po,173.000000
2219,Fa,356.000000


**Conclusion for Basement Has Area but BsmtQual is Missing**

According to the data documentation, BsmtQual specifically measures basement height in inches, not general quality.

  Analysis:
  - Both houses are very old (1895, 1910) with completely unfinished basements
  - ID 2218: Stone foundation, BsmtCond = Fa, suggesting an older, lower-quality basement
  - ID 2219: Poured concrete foundation, BsmtCond = TA, indicating slightly better construction

  Historical Context: Houses built in the 1890s-1910s typically had low basement ceilings, often below modern standards, as basements were primarily
  used for utilities rather than living space.

  Decision:
  - ID 2218: Set BsmtQual = "Po" (Poor, <70 inches) - stone foundation and fair condition suggest very low ceiling height
  - ID 2219: Set BsmtQual = "Fa" (Fair, 70-79 inches) - poured concrete foundation and typical condition suggest marginally better ceiling height

  This correction maintains logical consistency while reflecting realistic basement height expectations for houses of this vintage.

### 5.4 Verify Data Quality Corrections
Verify that the data quality corrections have been applied correctly and that the issues have been resolved.

In [99]:
# Display the dataframe with the issue after fixing

display(df_issues_sorted.style.hide(axis="index"))

Id,Issue,IssueCount


All issues have been addressed and the dataset is now cleaned up.

## 6. Data Type Optimization

Convert float columns to integer types where appropriate to improve model performance and reduce memory usage. This optimization ensures better
numerical stability during training while maintaining data integrity.

### 6.1 Data Type Analysis

Identify current data types

In [100]:
# Analyze current data types
print("Current Data Types Analysis:")
print(f"Dataset shape: {df_combined.shape}")

print(f"\nData type distribution:")
print(df_combined.dtypes.value_counts())

Current Data Types Analysis:
Dataset shape: (2917, 81)

Data type distribution:
object      44
int64       23
float64     11
category     1
category     1
category     1
Name: count, dtype: int64


The dataset contains 29 numberic features, of which 6 are float and 23 are integer types.

### 6.2 Float Features Verification

Check which float features actually contain whole numbers vs decimals

In [101]:
# Get all float features for analysis
float_features = df_combined.select_dtypes(include=['float64']).columns.tolist()

print("Float Features Analysis:")

whole_number_features = []
decimal_features = []

for feature in float_features:
    # Check if all values are whole numbers
    is_whole = (df_combined[feature] == df_combined[feature].astype(int)).all()

    print(f"\n{feature}:")
    print(f"  Range: {df_combined[feature].min():.2f} - {df_combined[feature].max():.2f}")
    print(f"  All whole numbers: {is_whole}")

    if not is_whole:
        decimal_vals = df_combined[feature][df_combined[feature] != df_combined[feature].astype(int)]
        print(f"  Sample decimal values: {decimal_vals.head(3).values}")
        decimal_features.append(feature)
    else:
        whole_number_features.append(feature)

print(f"\nSummary:")
print(f"Features safe for int conversion: {len(whole_number_features)}")
print(f"Features with decimals: {len(decimal_features)}")
print(f"\nWhole number features: {whole_number_features}")
print(f"Decimal features: {decimal_features}")


Float Features Analysis:

LotFrontage:
  Range: 21.00 - 313.00
  All whole numbers: False
  Sample decimal values: [66.5 73.5 66.5]

MasVnrArea:
  Range: 0.00 - 1600.00
  All whole numbers: True

BsmtFinSF1:
  Range: 0.00 - 4010.00
  All whole numbers: True

BsmtFinSF2:
  Range: 0.00 - 1526.00
  All whole numbers: True

BsmtUnfSF:
  Range: 0.00 - 2336.00
  All whole numbers: True

TotalBsmtSF:
  Range: 0.00 - 5095.00
  All whole numbers: True

BsmtFullBath:
  Range: 0.00 - 3.00
  All whole numbers: True

BsmtHalfBath:
  Range: 0.00 - 2.00
  All whole numbers: True

GarageYrBlt:
  Range: 0.00 - 2010.00
  All whole numbers: True

GarageCars:
  Range: 0.00 - 5.00
  All whole numbers: True

GarageArea:
  Range: 0.00 - 1488.00
  All whole numbers: True

Summary:
Features safe for int conversion: 10
Features with decimals: 1

Whole number features: ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

The float feature analysis reveals that 5 out of 6 float features contain whole numbers, while 1 feature (LotFrontage) have decimal values.

### 6.2 Convert float features to int where appropriate

In [102]:
# Convert float64 columns with only whole numbers to int
df_combined[whole_number_features] = df_combined[whole_number_features].astype('int')


### 6.5 Data Type Validation
Verify that all conversions completed successfully and data integrity is maintained.

In [103]:
# Validate data type conversions
print("Data Type Conversion Validation:")
print("="*45)

# Check that no missing values were introduced
print("Validation checks:")
null_check = df_combined.isnull().sum().sum()
print(f"  Total null values: {null_check}")

# Verify converted features
print(f"\nConverted features verification:")
for feature in whole_number_features:
    if feature in df_combined.columns:
        print(f"  {feature}: {df_combined[feature].dtype} | Range: {df_combined[feature].min()}-{df_combined[feature].max()}")

# Show final data type distribution
print(f"\nFinal data type distribution:")
print(df_combined.dtypes.value_counts())

Data Type Conversion Validation:
Validation checks:
  Total null values: 0

Converted features verification:
  MasVnrArea: int64 | Range: 0-1600
  BsmtFinSF1: int64 | Range: 0-4010
  BsmtFinSF2: int64 | Range: 0-1526
  BsmtUnfSF: int64 | Range: 0-2336
  TotalBsmtSF: int64 | Range: 0-5095
  BsmtFullBath: int64 | Range: 0-3
  BsmtHalfBath: int64 | Range: 0-2
  GarageYrBlt: int64 | Range: 0-2010
  GarageCars: int64 | Range: 0-5
  GarageArea: int64 | Range: 0-1488

Final data type distribution:
object      44
int64       33
category     1
float64      1
category     1
category     1
Name: count, dtype: int64


All the float features that contained whole numbers have been converted to integer types, while the LotFrontage feature remains as float due to its decimal values.

## 7. Data Validation and Export

Final validation and preparation of clean datasets for feature engineering phase.

### 7.1 Pipeline Validation

Validate that all preprocessing steps completed successfully across the entire pipeline.

In [104]:
# Final comprehensive validation
print("Final Preprocessing Pipeline Validation:")

# 1. Missing values check
total_missing = df_combined.isnull().sum().sum()
print(f"1. Missing Values Check:")
print(f"   Total missing values: {total_missing}")

# DEBUG: Show which features still have missing values
if total_missing > 0:
    missing_features = df_combined.isnull().sum()
    missing_features = missing_features[missing_features > 0]
    print(f"   Features with missing values:")
    for feature, count in missing_features.items():
        print(f"     {feature}: {count}")

# 2. Data type consistency
print(f"\n2. Data Type Consistency:")
print(f"   Data type distribution:")
dtype_counts = df_combined.drop('dataset_source', axis=1).dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"     {dtype}: {count} features")

# 3. Feature count validation
original_feature_count = 80  # 81 - 1 (Id removed)
current_feature_count = len(df_combined.columns) - 1  # -1 for dataset_source
print(f"\n3. Feature Count Validation:")
print(f"   Original features: {original_feature_count}")
print(f"   Current features: {current_feature_count}")

# 4. Outlier removal confirmation
train_rows = len(df_combined[df_combined['dataset_source'] == 'train'])
expected_train_rows = 1458  # 1460 - 2 outliers
print(f"\n4. Outlier Removal Confirmation:")
print(f"   Current train rows: {train_rows}")
print(f"   Expected train rows: {expected_train_rows}")

Final Preprocessing Pipeline Validation:
1. Missing Values Check:
   Total missing values: 0

2. Data Type Consistency:
   Data type distribution:
     object: 43 features
     int64: 33 features
     category: 1 features
     float64: 1 features
     category: 1 features
     category: 1 features

3. Feature Count Validation:
   Original features: 80
   Current features: 80

4. Outlier Removal Confirmation:
   Current train rows: 1458
   Expected train rows: 1458


The preprocessing pipeline has been validated with no missing values, consistent data types, preserved feature counts, and proper outlier removal.

### 7.2 Train/Test Consistency Validation

Ensure preprocessing consistency between training and test datasets.

In [105]:
# Split back to train/test for consistency validation
train_mask = df_combined['dataset_source'] == 'train'
test_mask = df_combined['dataset_source'] == 'test'

df_train_clean = df_combined[train_mask].drop('dataset_source', axis=1).copy()
df_test_clean = df_combined[test_mask].drop('dataset_source', axis=1).copy()

# Add SalePrice back to training data from original dataset
df_train_clean['SalePrice'] = df_train['SalePrice'].values

print("Train/Test Consistency Validation:")
# 1. Feature consistency (excluding SalePrice which is only in train)
train_features = set(df_train_clean.columns) - {'SalePrice'}
test_features = set(df_test_clean.columns)

print(f"1. Feature Set Consistency:")
if train_features == test_features:
    print("   Train and test have identical features (excluding target)")
else:
    missing_in_test = train_features - test_features
    missing_in_train = test_features - train_features
    if missing_in_test:
        print(f"   Features missing in test: {missing_in_test}")
    if missing_in_train:
        print(f"   Features missing in train: {missing_in_train}")

# 2. Data type consistency (excluding SalePrice)
print(f"\n2. Data Type Consistency:")
dtype_mismatches = []
for feature in train_features.intersection(test_features):
    if df_train_clean[feature].dtype != df_test_clean[feature].dtype:
        dtype_mismatches.append(feature)

if not dtype_mismatches:
    print("   All features have consistent data types")
else:
    print(f"   Data type mismatches: {dtype_mismatches}")
    for feature in dtype_mismatches:
        print(f"     {feature}: train={df_train_clean[feature].dtype}, test={df_test_clean[feature].dtype}")

print(f"\nDataset Shapes:")
print(f"   Clean train: {df_train_clean.shape}")
print(f"   Clean test: {df_test_clean.shape}")

Train/Test Consistency Validation:
1. Feature Set Consistency:
   Train and test have identical features (excluding target)

2. Data Type Consistency:
   All features have consistent data types

Dataset Shapes:
   Clean train: (1458, 81)
   Clean test: (1459, 80)


The training and test datasets have been validated for consistency in features and data types.

### 7.4 Export Clean Datasets

Export preprocessed datasets for feature engineering phase.

In [106]:
# Export clean datasets for Notebook 03
print("Exporting Clean Datasets:")

output_dir = "../data/processed/"


# Export training data
df_train_clean.to_csv(f"{output_dir}train_cleaned.csv", index=False)
print(f" Training data exported: {output_dir}train_cleaned.csv")

# Export test data
df_test_clean.to_csv(f"{output_dir}test_cleaned.csv", index=False)
print(f" Test data exported: {output_dir}test_cleaned.csv")

Exporting Clean Datasets:
 Training data exported: ../data/processed/train_cleaned.csv
 Test data exported: ../data/processed/test_cleaned.csv
